In [17]:
# First have qubit couple to resonator


import scqubits as scq
import qutip as qt
import numpy as np


EC = 0.1916
EJ = 10.96 # 0.22 * 110
E_osc = 7.425  # GHz, resonator frequency

tmon = scq.Transmon(
    EJ=EJ,      # Josephson energy
    EC=EC,      # Charging energy
    ng=0,     # Offset charge
    ncut=41,    # Charge basis cutoff
    truncated_dim=10  # Number of levels to keep
)


resonator = scq.Oscillator(E_osc=E_osc, truncated_dim=20)

hilbertspace = scq.HilbertSpace([tmon, resonator])

g = -0.14*1 #g = 120 MHz, calculate this from  capacitance matrix

hilbertspace.add_interaction(
    expr="g * n * a - g * n * adag",
    op1=("n", tmon.n_operator),
    op2=("adag", resonator.creation_operator),
    op3 = ("a", resonator.annihilation_operator),
    add_hc=False
)

hilbertspace.generate_lookup()
    
    # Extract eigenvalues
evals = hilbertspace["evals"][0]
diag_dressed_hamiltonian = (
    2 * np.pi * qt.Qobj(np.diag(evals), dims=[hilbertspace.subsystem_dims] * 2)
    )

total_truncation = 10
def truncate(operator: qt.Qobj, dimension: int) -> qt.Qobj:
        return qt.Qobj(operator[:dimension, :dimension])
    
diag_dressed_hamiltonian_trunc = truncate(diag_dressed_hamiltonian, total_truncation)
evalues = (diag_dressed_hamiltonian_trunc.eigenenergies() - diag_dressed_hamiltonian_trunc.eigenenergies()[0]) / 6.28
    
    # Compute chi
e_11 = evalues[hilbertspace.dressed_index((1,1))]
e_10 = evalues[hilbertspace.dressed_index((1,0))]
e_01 = evalues[hilbertspace.dressed_index((0,1))]
e_00 = evalues[hilbertspace.dressed_index((0,0))]
    
chi = e_11 - e_10 - e_01 + e_00



print(f"{chi*1000:.4f} Chi (MHz)")

# print(f"{e_11:.4f}, {e_10:.4f}, {e_01:.4f}, {e_00:.4f} Energies (GHz) for (11, 10, 01, 00) states")
detuning = e_10 - e_01
# print(f"{detuning:.4f} Detuning (GHz)")


Chi = abs(chi)

# Assume tmon.truncated_dim and resonator.truncated_dim are the subsystem dimensions

H_full = 2 * np.pi * hilbertspace.hamiltonian()
evals, ekets = H_full.eigenstates()

ekets_flat = np.array(ekets).flatten()
trunc_tmon = tmon.truncated_dim
trunc_res = resonator.truncated_dim

def get_dressed_epairs(evals, ekets):
    dressed_epairs = np.empty((trunc_tmon, trunc_res), dtype=object)
    for i in range(trunc_tmon):
        for j in range(trunc_res):
            index = hilbertspace.dressed_index((i, j))
            basis_vec = qt.tensor(qt.basis(trunc_tmon, i), qt.basis(trunc_res, j))
            ztemp = basis_vec.overlap(ekets[index])
            dressed_epairs[i, j] = (evals[index], np.exp(-1j * np.angle(ztemp)) * ekets[index])
    return dressed_epairs

dressed_epairs = get_dressed_epairs(evals, ekets_flat)


n_tmon_full = scq.identity_wrap(tmon.n_operator, tmon, [tmon, resonator])
# Use the correct indices and ensure dressed_epairs[...] are Qobj
n_tmon_00_01 = n_tmon_full.matrix_element(dressed_epairs[0, 0][1], dressed_epairs[0, 1][1])
n_tmon_10_11 = n_tmon_full.matrix_element(dressed_epairs[1, 0][1], dressed_epairs[1, 1][1])

print(f"{n_tmon_00_01:.4f} n_tmon_00_01")
print(f"{n_tmon_10_11:.4f} n_tmon_10_11")

1.4630 Chi (MHz)


TypeError: cannot calculate overlap with non-quantum object array([[Quantum object: dims=[[10, 20], [1, 1]], shape=(200, 1), type='ket', dtype=Dense
        Qobj data =
        [[ 9.99902910e-01]
         [-2.26730677e-19]
         [ 2.09322416e-04]
         [-6.87189635e-21]
         [ 5.39925056e-08]
         [-1.50231148e-21]
         [ 1.47649283e-11]
         [ 1.66540538e-22]
         [ 4.16001811e-15]
         [ 1.18984782e-24]
         [ 1.19397155e-18]
         [-2.98818686e-25]
         [ 3.46409235e-22]
         [-7.82513988e-26]
         [ 1.10553006e-25]
         [ 1.58394600e-26]
         [-9.27722979e-27]
         [ 2.39049924e-27]
         [ 1.38546811e-28]
         [-2.96208561e-28]
         [-1.44299523e-19]
         [-1.39228180e-02]
         [ 5.77245210e-20]
         [-4.98645553e-06]
         [-2.19054517e-23]
         [-1.64106887e-09]
         [ 9.61116130e-24]
         [-5.25041260e-13]
         [-1.18104239e-24]
         [-1.65831391e-16]
         [-1.38642240

In [46]:
# Assume tmon.truncated_dim and resonator.truncated_dim are the subsystem dimensions
trunc_tmon = tmon.truncated_dim
trunc_res = resonator.truncated_dim

def get_dressed_epairs(evals, ekets):
    dressed_epairs = np.empty((trunc_tmon, trunc_res), dtype=object)
    for i in range(trunc_tmon):
        for j in range(trunc_res):
            index = hilbertspace.dressed_index((i, j))
            basis_vec = qt.tensor(qt.basis(trunc_tmon, i), qt.basis(trunc_res, j))
            ztemp = basis_vec.overlap(ekets[index])
            dressed_epairs[i, j] = (evals[index], np.exp(-1j * np.angle(ztemp)) * ekets[index])
    return dressed_epairs

dressed_epairs = get_dressed_epairs(evals, ekets)

# Use the correct indices and ensure dressed_epairs[...] are Qobj
n_tmon_00_01 = n_tmon_full.matrix_element(dressed_epairs[0, 0][1], dressed_epairs[0, 1][1])
n_tmon_10_11 = n_tmon_full.matrix_element(dressed_epairs[1, 0][1], dressed_epairs[1, 1][1])

print(f"{n_tmon_00_01:.4f} n_tmon_00_01")
print(f"{n_tmon_10_11:.4f} n_tmon_10_11")

-0.0000-0.0372j n_tmon_00_01
0.0000-0.0262j n_tmon_10_11


In [ ]:

a_resonator_full = scq.identity_wrap(resonator.annihilation_operator, resonator, [tmon, resonator])
adag_resonator_full = scq.identity_wrap(resonator.creation_operator, resonator, [tmon, resonator])

n_resonator_full = a_resonator_full-adag_resonator_full

n_resonator_full.matrix_element(dressed_epairs[0, 0][1], dressed_epairs[0, 1][1])
n_resonator_full.matrix_element(dressed_epairs[1, 0][1], dressed_epairs[1, 1][1])
print(f"{n_resonator_full.matrix_element(dressed_epairs[0, 1][1], dressed_epairs[1, 1][1]):.4f} n_resonator_01_11")
print(f"{n_resonator_full.matrix_element(dressed_epairs[0, 0][1], dressed_epairs[1, 0][1]):.4f} n_resonator_00_10")

-0.0000-0.0419j n_resonator_01_11
0.0000-0.0419j n_resonator_00_10
